In [ ]:
]st

In [ ]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

In [ ]:
#]activate .
#]instantiate

In [ ]:
using Statistics, LinearAlgebra, DelimitedFiles

In [ ]:
Pkg.add("Gaston")
Pkg.add("StatsBase")

In [ ]:
### Uncomment line below to install Gaston.jl and StatsBase.jl if they're not installed yet

# import Pkg; Pkg.add("Gaston"); Pkg.add("StatsBase")

using Gaston, StatsBase

In [ ]:
res_radec_OR6 = readdlm("optical_residuals_OR6.txt")
res_radec_OR7 = readdlm("optical_residuals_OR7.txt")

res_α_OR6 = res_radec_OR6[:,1]
res_δ_OR6 = res_radec_OR6[:,2]

res_α_OR7 = res_radec_OR7[:,1]
res_δ_OR7 = res_radec_OR7[:,2]
;

# Figure 1: optical astrometry post-fit residuals

In [ ]:
#bins_hist = -2.0:0.1:2.0
nbins_plot = 100

#h_a = fit(Histogram, res_α_OR6, bins_hist)
h_a = fit(Histogram, res_α_OR6, nbins=nbins_plot)
bc_a = midpoints(h_a.edges[1]) # bin centers
bs_a = fill(h_a.edges[1].step.hi, length(bc_a)) # bin sizes
;

#h_b = fit(Histogram, res_δ_OR6, bins_hist)
h_b = fit(Histogram, res_δ_OR6, nbins=nbins_plot)
bc_b = midpoints(h_b.edges[1]) # bin centers
bs_b = fill(h_b.edges[1].step.hi, length(bc_b)) # bin sizes

#h_c = fit(Histogram, res_α_OR7, bins_hist)
h_c = fit(Histogram, res_α_OR7, nbins=nbins_plot)
bc_c = midpoints(h_c.edges[1]) # bin centers
bs_c = fill(h_c.edges[1].step.hi, length(bc_c)) # bin sizes

#h_d = fit(Histogram, res_δ_OR7, bins_hist)
h_d = fit(Histogram, res_δ_OR7, nbins=nbins_plot)
bc_d = midpoints(h_d.edges[1]) # bin centers
bs_d = fill(h_d.edges[1].step.hi, length(bc_d)) # bin sizes
;

In [ ]:
angv = vcat(range(0, length=1000, stop=prevfloat(2pi)), 0)
sigma_factor = 3.0
σ_cos_ang = sigma_factor*cos.(angv)
σ_sin_ang = sigma_factor*sin.(angv)

cov_αδ_cholf_OR6 = cholesky(cov(hcat(res_α_OR6, res_δ_OR6)))
cov_αδ_cholf_OR7 = cholesky(cov(hcat(res_α_OR7, res_δ_OR7)))

ell_3σ_x_OR6 = mean(res_α_OR6) .+ cov_αδ_cholf_OR6.L[1,1].*σ_cos_ang .+ cov_αδ_cholf_OR6.L[1,2].*σ_sin_ang
ell_3σ_y_OR6 = mean(res_δ_OR6) .+ cov_αδ_cholf_OR6.L[2,1].*σ_cos_ang .+ cov_αδ_cholf_OR6.L[2,2].*σ_sin_ang

ell_3σ_x_OR7 = mean(res_α_OR7) .+ cov_αδ_cholf_OR7.L[1,1].*σ_cos_ang .+ cov_αδ_cholf_OR7.L[1,2].*σ_sin_ang
ell_3σ_y_OR7 = mean(res_δ_OR7) .+ cov_αδ_cholf_OR7.L[2,1].*σ_cos_ang .+ cov_αδ_cholf_OR7.L[2,2].*σ_sin_ang
;

In [ ]:
gap = 0.01
xlim_plt = 3.5
ymin_plt = -5.0
ymax_plt = 5.0

Gaston.set(preamble="set grid; set style circle radius 0.02")

p1 = Gaston.plot(
    bc_a,
    h_a.weights./2, 
    supp=[bs_a./2 h_a.weights./2], 
    curveconf="w boxxy notit fs transparent solid 0.4", 
    Axes(
        #linetype = :tab10,
        axesconf="set lmargin at screen 0.1; set rmargin at screen 0.85; set tmargin at screen 0.99; set bmargin at screen 0.8+$gap",
        xrange = (-xlim_plt, xlim_plt),
        format = "''",
        label="1 '(b)' at graph 0.05,0.8"
    ),
    handle = 1
)
Gaston.plot!(
    bc_c,
    h_c.weights./2, 
    supp=[bs_c./2 h_c.weights./2], 
    curveconf="w boxxy notit fs transparent solid 0.4"
)

p2 = Gaston.plot(
    res_α_OR6, 
    res_δ_OR6, 
    curveconf="w circles fill transparent solid 0.4 noborder tit 'gravity-only'",
    Axes(
        #linetype = :tab10,
        key="at screen 0.835, screen 0.78 box spacing 1.2 height 0.4",  #"outside top right box spacing 1.2 height 1",
        axesconf="set lmargin at screen 0.1; set rmargin at screen 0.85; set tmargin at screen 0.8; set bmargin at screen 0.1",
        xrange = (-xlim_plt, xlim_plt),
        yrange = (ymin_plt, ymax_plt),
        format = "\"%g\"",
        xlabel="'\$\\alpha\\cos(\\delta)\$ \$[\$arcsec\$]\$'",
        ylabel="'\$\\delta\$ \$[\$arcsec\$]\$'",
        label="1 '(a)' at graph 0.05,0.925"
    ), 
    handle = 2
)
Gaston.plot!(res_α_OR7, res_δ_OR7, 
    curveconf="w circles fill transparent solid 0.4 noborder tit 'non-grav.'"
)
Gaston.plot!([mean(res_α_OR6)], [mean(res_δ_OR6)], curveconf="w p pt 4 ps 1.5 lw 8 notit") # tit 'mean R.A./Dec (OR6)'
Gaston.plot!([mean(res_α_OR7)], [mean(res_δ_OR7)], curveconf="w p pt 6 ps 1.5 lw 8 notit") # tit 'mean R.A./Dec (OR7)'
Gaston.plot!(ell_3σ_x_OR6, ell_3σ_y_OR6, curveconf="w l lw 8 lc 3 notit")
Gaston.plot!(ell_3σ_x_OR7, ell_3σ_y_OR7, curveconf="w l lw 8 lc 4 notit")

p4 = Gaston.plot(
    h_b.weights./2,
    bc_b,
    supp=[h_b.weights./2 bs_b./2],
    curveconf="w boxxy notit fs transparent solid 0.4",
    Axes(
        #linetype = :tab10,
        axesconf="set lmargin at screen 0.85+$gap; set rmargin at screen 0.99; set tmargin at screen 0.8; set bmargin at screen 0.1",
        yrange = (ymin_plt, ymax_plt),
        format = "''",
        label="1 '(c)' at graph 0.15,0.925"
    ),
    handle = 4
)
Gaston.plot!(
    h_d.weights./2,
    bc_d,
    supp=[h_d.weights./2 bs_d./2],
    curveconf="w boxxy notit fs transparent solid 0.4"
)

Gaston.plot([p1 nothing ; p2 p4])

#Gaston.save(term = "cairolatex", output = "fig1.tex", saveopts = "pdf standalone color dashed transparent size 7in,5in font ',12'")
#run(`lualatex fig1.tex`)

In [ ]:
Gaston.save(term = "cairolatex", output = "fig1.tex", saveopts = "pdf standalone color dashed transparent size 7in,5in font 'Arial,12'")
run(`lualatex fig1.tex`)

# Figure 2: radar astrometry post-fit residuals

In [ ]:
# This is the data that will be plotted in Figure 2; it was computed with the procedure described in Apophis.ipynb

deldop_dates_plot_xaxis = ["01/06/04 0000", "29/01/05 0000", "30/01/05 0018", "22/12/12 1100", "03/01/13 0920", "03/01/13 1000", "03/01/13 1050", "05/01/13 1040", "05/01/13 1050", "09/01/13 0800", "10/01/13 0800", "10/01/13 0940", "14/01/13 0810", "14/01/13 0950", "16/01/13 0630", "18/02/13 0137", "20/02/13 0126", "15/03/13 2359", "05/03/21 0700", "08/03/21 0250", "11/03/21 0820", "31/03/21 2359"]
del_dates_plot = ["29/01/05 0000", "30/01/05 0018", "22/12/12 1100", "03/01/13 0920", "03/01/13 1000", "03/01/13 1050", "05/01/13 1040", "05/01/13 1050", "09/01/13 0800", "10/01/13 0800", "10/01/13 0940", "14/01/13 0810", "14/01/13 0950", "16/01/13 0630", "18/02/13 0137", "20/02/13 0126", "15/03/13 2359", "05/03/21 0700", "08/03/21 0250", "11/03/21 0820"]
res_τ_x_OR6 = [-205.7663483595101, -209.607912779036, -6.679602473002039, 2.7688156896429237, 2.2728645789324062, 2.7989098640418257, 2.8526849747399075, 2.377649842966868, 1.8883925268438535, 1.2086849385385565, 1.1668566429561231, -2.5681211024235107, -2.594771592572707, -5.333688042882464, -2.6968265136229777, 3.3562808802307886, 127.69413487317352, -2.518584273719057, -13.123348131483908, -19.771368667742536]
res_τ_x_OR7 = [0.050396443081149433, -0.025433076986406622, -0.0014840203032733257, -0.0020486441839971947, -0.50886473233698, 0.006298170810781045, -0.13534148193220655, -0.610371419081909, 0.08311202454337113, 0.031976781638244134, 0.02824151365185068, -0.03589296925508112, 0.009588150784925475, -0.45739581443411365, -0.09899766007215002, 0.07679700821359997, 0.3861261932189487, 0.6545019445480591, -0.2726697925310759, -0.4029401725617845]
σ_del_all = [4.0, 4.5, 0.25, 0.25, 3.0, 0.25, 0.25, 3.0, 0.2, 0.2, 0.2, 0.2, 0.2, 1.0, 0.25, 0.25, 2.0, 1.0, 1.0, 1.0]
;

In [ ]:
tmargin_bmargin_gap = 0.03
bmargin_ths_plt = 0.5 + tmargin_bmargin_gap
tmargin_ths_plt = 0.965
axsbrk_v = 1.25*0.01
axsbrk_h = 1.25*0.005
lw_ths_plt = 3
rmargin_ths_plt = 0.3

q1 = Gaston.plot(
    del_dates_plot,
    res_τ_x_OR6,
    supp = σ_del_all,
    curveconf = "u 1:3:4 w errorbars pt 6 lw $lw_ths_plt notit",
    Axes(
        axesconf = "set rmargin at screen 0.3; set tmargin at screen $tmargin_ths_plt; set bmargin at screen $bmargin_ths_plt",
        xdata   = "time",
        timefmt = "'%d/%m/%y %H%M'",
        xtics = false,
        xrange = "[\"01/12/04\":\"30/05/06\"]",
        bars = false,
        ylabel = "'Time-delay resid. (O-C) [\$\\mu\$s]'",
        border = "1+2+4",
        ytics = ["nomirror", "format '%1.1f'"],
        yrange = "[-8.0:6.0]",
        arrow = [
            "from screen 0.3-$axsbrk_h,$bmargin_ths_plt-$axsbrk_v to screen 0.3+$axsbrk_h,$bmargin_ths_plt+$axsbrk_v nohead",
            "from screen 0.3-$axsbrk_h,$tmargin_ths_plt-$axsbrk_v to screen 0.3+$axsbrk_h,$tmargin_ths_plt+$axsbrk_v nohead"
            ],
        label="1 '(a)' at graph 0.2,0.9",
        grid=false
    ),
    handle = 1
)
Gaston.plot!(
    del_dates_plot,
    res_τ_x_OR7,
    supp = σ_del_all,
    curveconf = "u 1:3:4 w errorbars pt 4 lw $lw_ths_plt notit",
    lc = 2
)
Gaston.plot!(deldop_dates_plot_xaxis, zeros(length(deldop_dates_plot_xaxis)), 
    curveconf=" u 1:3 w l lc rgb 'gray' lw 4 dt 2 notit")

q2 = Gaston.plot(
    del_dates_plot,
    res_τ_x_OR6,
    supp = σ_del_all,
    curveconf = "u 1:3:4 w errorbars pt 6 lw $lw_ths_plt notit",
    Axes(
        xdata   = "time",
        timefmt = "'%d/%m/%y %H%M'",
        axesconf = "set lmargin at screen 0.35; set tmargin at screen $tmargin_ths_plt; set rmargin at screen 0.8; set bmargin at screen $bmargin_ths_plt",
        xtics = false,
        xrange = "[\"15/12/12\":\"20/03/13\"]",
        bars = false,
        ytics = false,
        border = "1+4",
        yrange = "[-8.0:6.0]",
        format = ["x ''", "y ''"],
        arrow = [
            "from screen 0.35-$axsbrk_h,$bmargin_ths_plt-$axsbrk_v to screen 0.35+$axsbrk_h,$bmargin_ths_plt+$axsbrk_v nohead",
            "from screen 0.35-$axsbrk_h,$tmargin_ths_plt-$axsbrk_v to screen 0.35+$axsbrk_h,$tmargin_ths_plt+$axsbrk_v nohead",
            "from screen 0.80-$axsbrk_h,$bmargin_ths_plt-$axsbrk_v to screen 0.80+$axsbrk_h,$bmargin_ths_plt+$axsbrk_v nohead",
            "from screen 0.80-$axsbrk_h,$tmargin_ths_plt-$axsbrk_v to screen 0.80+$axsbrk_h,$tmargin_ths_plt+$axsbrk_v nohead"
            ],
    ),
    handle = 2
)
Gaston.plot!(
    del_dates_plot,
    res_τ_x_OR7,
    supp = σ_del_all,
    curveconf = "u 1:3:4 w errorbars pt 4 lw $lw_ths_plt notit",
    lc = 2
)
Gaston.plot!(deldop_dates_plot_xaxis, zeros(length(deldop_dates_plot_xaxis)), 
    curveconf=" u 1:3 w l lc rgb 'gray' lw 4 dt 2 notit")

q3 = Gaston.plot(
    del_dates_plot,
    res_τ_x_OR6,
    supp = σ_del_all,
    curveconf = "u 1:3:4 w errorbars pt 6 lw $lw_ths_plt notit",
    Axes(
        xdata   = "time",
        timefmt = "'%d/%m/%y %H%M'",
        axesconf = "set lmargin at screen 0.85; set tmargin at screen 0.965; set bmargin at screen $bmargin_ths_plt",
        xtics = false,
        xrange = "[\"02/03/21\":\"15/03/21\"]",
        bars = false,
        ytics = false,
        border = "1+4+8",
        yrange = "[-8.0:6.0]",
        format = ["x ''", "y ''"],
        arrow = [
            "from screen 0.85-$axsbrk_h,$bmargin_ths_plt-$axsbrk_v to screen 0.85+$axsbrk_h,$bmargin_ths_plt+$axsbrk_v nohead",
            "from screen 0.85-$axsbrk_h,$tmargin_ths_plt-$axsbrk_v to screen 0.85+$axsbrk_h,$tmargin_ths_plt+$axsbrk_v nohead"
            ],
    ),
    handle = 3
)
Gaston.plot!(
    del_dates_plot,
    res_τ_x_OR7,
    supp = σ_del_all,
    curveconf = "u 1:3:4 w errorbars pt 4 lw $lw_ths_plt notit",
    lc = 2
)
Gaston.plot!(deldop_dates_plot_xaxis, zeros(length(deldop_dates_plot_xaxis)), 
    curveconf=" u 1:3 w l lc rgb 'gray' lw 4 dt 2 notit")

Gaston.plot([q1 q2 q3])

In [ ]:
dop_dates_plot = ["27/01/05 2331", "29/01/05 0000", "30/01/05 0018", "07/08/05 1707", "06/05/06 1249", "21/12/12 1110", "22/12/12 1100", "03/01/13 0920", "05/01/13 0840", "05/01/13 1040", "06/01/13 0820", "06/01/13 0930", "08/01/13 0810", "09/01/13 0800", "09/01/13 0920", "10/01/13 0800", "11/01/13 0720", "14/01/13 0810", "15/01/13 0630", "16/01/13 0630", "16/01/13 0750", "17/01/13 0620", "18/02/13 0056", "18/02/13 0137", "19/02/13 0108", "20/02/13 0038", "20/02/13 0126", "21/02/13 0104", "15/03/13 2359", "03/03/21 0640"]
res_ν_x_OR6 = [0.022020119775167143, 0.09055169625891989, 0.13154706274193997, 1.060600940788269, 0.26066742865683734, -0.1091674378795128, -0.17515569383803867, -0.07166150341896857, 0.05517587752853472, -0.07363078865331714, -0.09369698279229627, 0.019300567103596195, -0.018171566429119036, -0.002313025763994163, 0.0018569626717937215, 0.05316121540304947, 0.13833776813270876, 0.019802340376966765, 0.07763852411908247, 0.00303957751020921, 0.16892640288076116, 0.08100276857549432, -0.11625174327216614, -0.1777097446627686, -0.12008464504963755, -0.11532927829238754, -0.10473141447818798, -0.1031180600672351, -0.07446039079671286, 0.627690483896775]
res_ν_x_OR7 = [-0.09610075436850195, -0.01928557305521883, 0.02973708786934942, -0.16476219049801466, -0.05413065416153498, 0.0025204031183949155, -0.05964928635627331, -0.029636496831002355, 0.07481118260222472, -0.07286090096609842, -0.08609909732825141, 0.017001626622561477, -0.038967132921936216, -0.03652170092533757, -0.042417022655732106, 0.004802233153849457, 0.08036206694888348, -0.07796196249108964, -0.021715853079137495, -0.10445135572225038, 0.05592466248110908, -0.03260433488501999, -0.042523788308621045, -0.10388362805207595, -0.041316116485705136, -0.031612410926241945, -0.020940257988383348, -0.014433378341653213, 0.11499911832261954, 0.07095373696789409]
σ_dop_all = [0.25, 0.25, 0.15, 0.2, 0.1, 0.25, 0.1, 0.1, 0.2, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.2, 0.1, 0.1, 0.2, 0.1, 0.1, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.238, 0.4]
;

In [ ]:
tmargin_ths_plt = 0.5 - tmargin_bmargin_gap
bmargin_ths_plt = 0.05

Gaston.set(preamble="unset grid")
q4 = Gaston.plot(
    dop_dates_plot,
    res_ν_x_OR6,
    supp = σ_dop_all,
    curveconf = "u 1:3:4 w errorbars pt 6 lw $lw_ths_plt notit",
    Axes(
        #linetype = :tab10,
        axesconf="set rmargin at screen $rmargin_ths_plt; set tmargin at screen $tmargin_ths_plt; set bmargin at screen $bmargin_ths_plt",
        xdata   = "time",
        timefmt = "'%d/%m/%y %H%M'",
        xtics = "format '%d/%b/%y' time (\"01/01/05\",\"01/01/06\")",
        xrange = "[\"01/12/04\":\"30/05/06\"]",
        #xlabel  = "\"Date\"",
        bars = false,#2,
        ylabel = "'Doppler shift resid. (O-C) [Hz]'",
        border="1+2+4",
        ytics = ["nomirror", "format '%1.1f'"],
        yrange="[-0.5:0.5]",
        arrow = [
            "from screen 0.3-$axsbrk_h,$tmargin_ths_plt-$axsbrk_v to screen 0.3+$axsbrk_h,$tmargin_ths_plt+$axsbrk_v nohead",
            "from screen 0.3-$axsbrk_h,$bmargin_ths_plt-$axsbrk_v to screen 0.3+$axsbrk_h,$bmargin_ths_plt+$axsbrk_v nohead"
            ],
        label="1 '(b)' at graph 0.2,0.9"
        #title = "'Doppler shift resid. (O-C)'"
    ),
    handle = 4
)
Gaston.plot!(
    dop_dates_plot,
    res_ν_x_OR7,
    supp = σ_dop_all,
    curveconf = "u 1:3:4 w errorbars pt 4 lw $lw_ths_plt notit",
    lc = 2
)
Gaston.plot!(deldop_dates_plot_xaxis, zeros(length(deldop_dates_plot_xaxis)), 
    curveconf=" u 1:3 w l lc rgb 'gray' lw 4 dt 2 notit")

q5 = Gaston.plot(
    dop_dates_plot,
    res_ν_x_OR6,
    supp = σ_dop_all,
    curveconf = "u 1:3:4 w errorbars pt 6 lw $lw_ths_plt notit",
    Axes(
        #linetype = :tab10,
        xdata   = "time",
        timefmt = "'%d/%m/%y %H%M'",
        axesconf="set lmargin at screen $rmargin_ths_plt+0.05; set rmargin at screen 0.8; set tmargin at screen $tmargin_ths_plt; set bmargin at screen $bmargin_ths_plt",
        xtics="format '%d/%b/%y' time (\"01/01/13\",\"01/02/13\",\"01/03/13\")",
        xrange="[\"15/12/12\":\"20/03/13\"]",
        #xlabel  = "\"Date\"",
        bars = false, #2,
        ytics=false,
        border="1+4",
        yrange="[-0.5:0.5]",
        arrow = [
            "from screen 0.35-$axsbrk_h,$tmargin_ths_plt-$axsbrk_v to screen 0.35+$axsbrk_h,$tmargin_ths_plt+$axsbrk_v nohead",
            "from screen 0.35-$axsbrk_h,$bmargin_ths_plt-$axsbrk_v to screen 0.35+$axsbrk_h,$bmargin_ths_plt+$axsbrk_v nohead",
            "from screen 0.8-$axsbrk_h,$tmargin_ths_plt-$axsbrk_v to screen 0.8+$axsbrk_h,$tmargin_ths_plt+$axsbrk_v nohead",
            "from screen 0.8-$axsbrk_h,$bmargin_ths_plt-$axsbrk_v to screen 0.8+$axsbrk_h,$bmargin_ths_plt+$axsbrk_v nohead"
            ],
        #ylabel = "'[Hz]'"
    ),
    handle = 5
)
Gaston.plot!(
    dop_dates_plot,
    res_ν_x_OR7,
    supp = σ_dop_all,
    curveconf = "u 1:3:4 w errorbars pt 4 lw $lw_ths_plt notit",
    lc = 2
)
Gaston.plot!(deldop_dates_plot_xaxis, zeros(length(deldop_dates_plot_xaxis)), 
    curveconf=" u 1:3 w l lc rgb 'gray' lw 4 dt 2 notit")

q6 = Gaston.plot(
    dop_dates_plot,
    res_ν_x_OR6,
    supp = σ_dop_all,
    curveconf = "u 1:3:4 w errorbars pt 6 lw $lw_ths_plt notit",
    Axes(
        #linetype = :tab10,
        xdata   = "time",
        timefmt = "'%d/%m/%y %H%M'",
        axesconf="set lmargin at screen 0.85; set tmargin at screen $tmargin_ths_plt; set bmargin at screen $bmargin_ths_plt",
        xtics="format '%d/%b/%y' time (\"03/03/21\",\"13/03/21\")",
        xrange="[\"02/03/21\":\"15/03/21\"]",
        #xlabel  = "\"Date\"",
        bars = false, #2,
        #format="y ''",
        ytics=false,
        border="1+4+8",
        yrange="[-0.5:0.5]",
        arrow = [
            "from screen 0.85-$axsbrk_h,$tmargin_ths_plt-$axsbrk_v to screen 0.85+$axsbrk_h,$tmargin_ths_plt+$axsbrk_v nohead",
            "from screen 0.85-$axsbrk_h,$bmargin_ths_plt-$axsbrk_v to screen 0.85+$axsbrk_h,$bmargin_ths_plt+$axsbrk_v nohead"
            ],
        #ylabel = "'[Hz]'"
    ),
    handle = 6
)
Gaston.plot!(
    dop_dates_plot,
    res_ν_x_OR7,
    supp = σ_dop_all,
    curveconf = "u 1:3:4 w errorbars pt 4 lw $lw_ths_plt notit",
    lc = 2
)
Gaston.plot!(deldop_dates_plot_xaxis, zeros(length(deldop_dates_plot_xaxis)), 
    curveconf=" u 1:3 w l lc rgb 'gray' lw 4 dt 2 notit")

Gaston.plot([q4 q5 q6])

In [ ]:
Gaston.plot([q1 q2 q3; q4 q5 q6])

#Gaston.save(term = "cairolatex", output = "fig2.tex", saveopts = "pdf standalone color dashed transparent size 10in,7in font ',14'")
#run(`lualatex fig2.tex`)

In [ ]:
Gaston.save(term = "cairolatex", output = "fig2.tex", saveopts = "pdf standalone color dashed transparent size 10in,7in font 'Arial,14'")
run(`lualatex fig2.tex`)

# Figure 3: $\zeta_{2029}$ B-plane

In [ ]:
# Earth radius
RE = 6378.1363

In [ ]:
gaussianpdf(x,μ=0.0,σ=1.0) = exp( -0.5((x-μ)/σ)^2 )/(σ*sqrt(2pi))

In [ ]:
B_2029_ζ_OR7_v15 = 7.389785726736688
B_2029_ζ_OR7_NEW = 7.425778700698029
σ_ζ_OR7_v15 = 0.047512928676830185
σ_ζ_OR7_NEW = 0.00024846584939182214

In [ ]:
xv_plt_hst = collect(7.2:0.00001:7.6)
yv_plt_hst_OR7_v15 = gaussianpdf.(xv_plt_hst,B_2029_ζ_OR7_v15, σ_ζ_OR7_v15)
yv_plt_hst_OR7_NEW = gaussianpdf.(xv_plt_hst,B_2029_ζ_OR7_NEW, σ_ζ_OR7_NEW)

ζ_2036_km = 46169.447125491504
ζ_2068_km = 49209.2286172537

In [ ]:
Gaston.plot(xv_plt_hst*RE, yv_plt_hst_OR7_v15/RE, #title="'OR7 (pre-2021)'", lw=4, 
    curveconf = "w l lw 4 tit 'OR7 (pre-2021)'",
    Axes(
        key="top right box opaque spacing 1.4 width 4.0 height 0.4",
        linetype = :tab10,
        yrange=(0,15/RE),
        xlabel="'\$\\zeta_{2029}\$ [km]'",
        ylabel="'Prob. density [km\$^{-1}\$]'",
        ytics="offset 0.75"
    ))
Gaston.plot!(xv_plt_hst*RE, yv_plt_hst_OR7_NEW/RE, #title="'OR7 (post-2021)'", lw=4,
    curveconf = "w l lw 4 tit 'OR7 (post-2021)'")
Gaston.plot!([ζ_2036_km,ζ_2036_km], [0,15]/RE, #title="'2036 return'", lw=4, dt=2)
    curveconf = "w l lw 4 dt 2 tit '2036 return'")

Gaston.plot!([ζ_2068_km,ζ_2068_km], [0,15]/RE, #title="'2068 return'", lw=4, dt=3)
    curveconf = "w l lw 4 dt 2 tit 'OR7 (pre-2021)'")
#Gaston.save(term = "cairolatex", output = "fig3.tex", saveopts = "pdf standalone color dashed transparent size 7in,5in font ',14'") ###
#run(`lualatex fig3.tex`)

In [ ]:
Gaston.save(term = "cairolatex", output = "fig3.tex", saveopts = "pdf standalone color dashed transparent size 7in,5in font 'Arial,14'") ###
run(`lualatex fig3.tex`)